In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
feature_names = ['party', 'handicapped-infants', 'water-project-cost-sharing',
                 'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                 'el-salvador-aid', 'religious-groups-in-schools',
                 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                 'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                 'education-spending', 'superfund-right-to-sue', 'crime',
                 'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv('house-votes-84.data.txt', na_values=['?'],
                          names=feature_names)

voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [3]:
voting_data.dropna(inplace=True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


In [4]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


In [5]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values
print(all_features[0])
print(all_classes[0])
print(len(all_classes))

[0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1]
1
232


In [6]:
train_features = all_features[:200]
train_result = all_classes[:200]
test_features = all_features[200:]
test_result = all_classes[200:]

In [7]:
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Method 1

In [8]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

In [9]:
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

0.9438405811786652

# Method 2

In [10]:
model = create_model()

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_features, train_result,
                    batch_size=25,
                    epochs=50,
                    validation_data=(test_features, test_result))

Epoch 1/50
8/8 [==============================] - 0s 15ms/step - loss: 0.6922 - accuracy: 0.6900 - val_loss: 0.6911 - val_accuracy: 0.8125
Epoch 2/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6892 - accuracy: 0.8650 - val_loss: 0.6878 - val_accuracy: 0.8438
Epoch 3/50
8/8 [==============================] - 0s 8ms/step - loss: 0.6843 - accuracy: 0.8850 - val_loss: 0.6818 - val_accuracy: 0.8750
Epoch 4/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6755 - accuracy: 0.8900 - val_loss: 0.6711 - val_accuracy: 0.8750
Epoch 5/50
8/8 [==============================] - 0s 8ms/step - loss: 0.6605 - accuracy: 0.8900 - val_loss: 0.6543 - val_accuracy: 0.8750
Epoch 6/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6382 - accuracy: 0.8900 - val_loss: 0.6301 - val_accuracy: 0.8750
Epoch 7/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6077 - accuracy: 0.8850 - val_loss: 0.5959 - val_accuracy: 0.8750
Epoch 8/50
8/8 [=================

In [12]:
score = model.evaluate(test_features, test_result)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1/1 [==============================] - 0s 980us/step - loss: 0.2528 - accuracy: 0.9062
Test loss: 0.25278350710868835
Test accuracy: 0.90625


In [13]:
features = np.reshape(all_features[0], (1, 16))
pred = model.predict(features)

if pred>=0.5:
    print("democrat")
else:
    print("reublican")

democrat
